This python notebook semi-automates the process of converting raw metagenomic pair-end sequencing data (.fastq.gz) to OTU & taxonomic table (.biom) and phylogenetic tree (.tre). The OTU & taxonomic table and phylogenetic tree files can then be used for diversity analysis such as making bar chart and PCoA. 

Reference documentation
Based on tutorial http://qiime.org/install/install.html https://nbviewer.jupyter.org/github/biocore/qiime/blob/1.9.1/examples/ipynb/illumina_overview_tutorial.ipynb

In [1]:
# name a working root folder for metagenomic analysis
folderName = 'zooplankton'

In [ ]:
# import relevant python library
import os

In [10]:
# go to working directory
os.chdir('/Users/pakpoomton/qiime/'+ folderName)

** manually** Unpack compressed raw data files (.fastq.gz) to raw data file (.fastq) 
Macrogen sequencing service provides you with one ".fastq.gz" file per one sample you sent.
(For HN00126595 sequencing round, we have 18 .fastq.gz files from 9 zooplankton samples x 2 reads (fwd and rwd))

** manually** Rename and organise .fastq file according to "multiple_join_paired_ends.py" required format
For examples, 
G1_R1_001.fastq
G1_R2_001.fastq
G2_R1_002.fastq
G2_R2_002.fastq
...
xx_R1_00n.fastq
xx_R2_00n.fastq

(For HN00126595 zooplankton, we have 18 .fastq file in total)

Put all these files in a folder "pairRead/" within our root folder "zooplankton/"

In [15]:
# join pair-end sequencing read files (.fastq) to a single file (.fastq)
!multiple_join_paired_ends.py -i pairRead/ -o joinedSeq/

After executing "multiple_join_paired_end.py", a folder "joinedSeq/" is created within our root folder "zooplankton/". The folder "joinedSeq/" has one folder inside for each joined fastq file. 

(For HN00126595 zooplankton, we have 9 folders in "joinedSeq" named after joined fastq files. 
 For examples, G1_R1_001/, G2_R1_002/, ...)
 
In each of these folders inside "joinedSeq/", we have three files:
fastqjoin.join.fastq, fastqjoin.un1.fastq, fastqjoin.un2.fastq

**manually**  Rename "fastqjoin,join.fastq" from each folder according to sample name and put them back to our root folder "zooplankton/"

(For HN00126595 zooplankton, we rename these files to G1.fastq, G2.fastq, ... . 
All these files are under zooplankton/ folder)

In [21]:
# filter sequencing results in each .fastq according to their quality and convert to .fna file
# ** need to change input-output and parameter names below according to file names 
# (G1, G2, G3, R1, R2, R3, S1, S2, S3)

# input -q determines Phred quality threshold for filtering sequencing read. 
!split_libraries_fastq.py -i G1.fastq --sample_ids G1 -o sloutG1_single_sample_q20/ -q 19 --barcode_type 'not-barcoded'

When we execute "split_libraries_fastq.py", a new folder "sloutXX_single_sample_q20/" is created inside our root directory "zooplankton/". Each of these folders has a file "seqs.fna" which as a list of joined sequence read without quality info. 

**Manually** rename "seqs.fna" from each folder according to sample name and put back in our root folder "zooplankton/"

(For HN00126595 zooplankton, we name these .fna files to: seqs_G1.fna, seqs_G2.fna, .... We have 9 files in total
from our 9 zooplankton samples.) 

In [6]:
# get OTU and phylogenic data from .fna file collection 
!pick_open_reference_otus.py -o otus/ -i seqs_G1.fna,seqs_G2.fna,seqs_G3.fna,seqs_R1.fna,seqs_R2.fna,seqs_R3.fna,seqs_S1.fna,seqs_S2.fna,seqs_S3.fna   -p uc_fast_params.txt



When we execute "pick_open_reference_otus.py" using all 9 .fna files as inputs with "uc_fast_params.txt" parameter,
we create a new folder "otus/" inside out root folder "zooplankton/"

Inside "otus/", we have the following files:

** OTU & taxonomic table **
otu_table_mc2.biom
otu_table_mc2_w_tax.biom
otu_table_mc2_w_tax_no_pynast_failures.biom   --> use this one

** phylogenetic tree **
rep_set.tre

(There are other files and folders which we don't use for now in our downstream analysis) 

In [8]:
# checking .biom file
!biom summarize-table -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom

Num samples: 9
Num observations: 3,204
Total count: 369,288
Table density (fraction of non-zero values): 0.190

Counts/sample summary:
 Min: 30,937.000
 Max: 49,047.000
 Median: 40,029.000
 Mean: 41,032.000
 Std. dev.: 5,593.843
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
G3: 30,937.000
G2: 35,841.000
G1: 36,598.000
S2: 39,956.000
R1: 40,029.000
R2: 44,301.000
R3: 45,685.000
S1: 46,894.000
S3: 49,047.000


In [9]:
# diversity analysis using data from .biom and .tre files. 
!core_diversity_analyses.py -o cdout/ -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom -m zooplankton-map.tsv -t otus/rep_set.tre -e 1114


/Users/pakpoomton/anaconda/envs/qiime1/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/Users/pakpoomton/anaconda/envs/qiime1/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


After executing "core_diversity_analyses.py", a folder "cdout/" is created. Inside this folder, a file "index.html" is a portal to all analysis results. 